# Training an Image Classification model for NACTI

Train a model on the exclusive empty vs animal dataframes

### Set up the environment

In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.utils.training_utils import multi_gpu_model
from keras.callbacks import ModelCheckpoint, TensorBoard

from collections import Counter

import pandas as pd

import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"]="0"

Using TensorFlow backend.


In [2]:
# path to the model weights files.
# weights_path = '../keras/examples/vgg16_weights.h5'
# top_model_weights_path = 'fc_model.h5'
# dimensions of our images.
img_width, img_height = 224, 224

train_dataframe_path = '/data/dataframes/exclusiveEVATrain.csv'
validation_data_dir = '/data/dataframes/exclusiveEVATest.csv'
checkpoint_dir = '/data/ResNet50/ResNet50_20190403_exclusiveEVA_weights_2.h5'
tensorboard_dir = '/data/ResNet50/exclusiveEva_2/logs'
nb_train_samples = 50000
nb_validation_samples = 10000
epochs = 25
batch_size = 48
# gpu_count = 2

### Import the dataframes

In [3]:
# Import the training and validation dataframes
train_df = pd.read_csv(train_dataframe_path)
val_df = pd.read_csv(validation_data_dir)

# Check to ensure that the camera trap locations are disjoint
assert len(train_df[train_df['cam_location'].isin(val_df['cam_location'].unique())]) == 0, "Train and validation are not disjoint"

# Get the number of classes
min(train_df['target'].nunique(), val_df['target'].nunique())

2

### Build the model

In [4]:
# build the MobileNetV2 network
ResNet50 = applications.ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
model = Sequential()
model.add(ResNet50)
model.add(Flatten(input_shape=model.output_shape[1:]))
model.add(Dense(196, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

# Make it a multi-gpu model if available
# model = multi_gpu_model(model, gpus=gpu_count)

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath=checkpoint_dir, verbose=1, save_best_only=True)
tboard = TensorBoard(tensorboard_dir)

Instructions for updating:
Colocations handled automatically by placer.


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model loaded.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### Create Data Generators

In [5]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    x_col='abs_file_path', 
    y_col ='target', 
    target_size=(img_width, img_height), 
    batch_size=batch_size, 
    shuffle=True,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_dataframe(
    val_df, 
    x_col='abs_file_path', 
    y_col ='target', 
    target_size=(img_width, img_height), 
    batch_size=batch_size, 
    shuffle=True,
    class_mode='categorical')

counter = Counter(train_generator.classes)                          
max_val = float(max(counter.values()))       
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}  

Found 169821 images belonging to 2 classes.
Found 48164 images belonging to 2 classes.


Define the weights

In [6]:
if max([v for k,v in class_weights.items()]) < 2:
    class_weights = {0:1, 1:1}

### Train the model

In [7]:
# fine-tune the model
history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples//batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples//batch_size, 
    class_weight=class_weights, 
    max_queue_size=batch_size*4,
    callbacks=[checkpointer, tboard])

Instructions for updating:
Use tf.cast instead.
Epoch 1/25
1041/1041 [==============================] - 807s 775ms/step - loss: 0.4729 - acc: 0.7811 - val_loss: 0.4246 - val_acc: 0.8046

Epoch 00001: val_loss improved from inf to 0.42458, saving model to /data/ResNet50/ResNet50_20190403_exclusiveEVA_weights_2.h5
Epoch 2/25
1041/1041 [==============================] - 795s 764ms/step - loss: 0.3473 - acc: 0.8396 - val_loss: 0.4123 - val_acc: 0.8220

Epoch 00002: val_loss improved from 0.42458 to 0.41235, saving model to /data/ResNet50/ResNet50_20190403_exclusiveEVA_weights_2.h5
Epoch 3/25
1041/1041 [==============================] - 797s 766ms/step - loss: 0.3135 - acc: 0.8557 - val_loss: 0.3772 - val_acc: 0.8277

Epoch 00003: val_loss improved from 0.41235 to 0.37725, saving model to /data/ResNet50/ResNet50_20190403_exclusiveEVA_weights_2.h5
Epoch 4/25
1041/1041 [==============================] - 794s 763ms/step - loss: 0.3017 - acc: 0.8626 - val_loss: 0.3660 - val_acc: 0.8349

Epoch 0